In [1]:
# Step1-build DB and create collections

#Import libraries
import pymongo
import json
from flask import jsonify
#import csv bring in if we need to pull in csv
#import pandas as pd if we use any data frames bring in

# Connect to the MongoDB server
client = pymongo.MongoClient('mongodb://localhost:27017/')
client.drop_database('healthbycountydata')
# Create or get the 'healthbycountydata' database
db = client['healthbycountydata']


ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it, Timeout: 30s, Topology Description: <TopologyDescription id: 64c07dcf5e9d90d515279891, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it')>]>

In [ ]:
# Step2 -Loading data sets.  All Data sets need to be located in folder on local 
#Data is in JSON format to load 
#JSON folder is titled Project_3_Data

In [ ]:
# json data file paths 
#if we have more data sources just add new line below. 
jsonfile_cancer = 'Cleaned_Data/cancer(2015-2019).json'
jsonfile_asthma = 'Cleaned_Data/asthma(2018-2020).json'
jsonfile_epa = 'Cleaned_Data/modified_final_epa_frs_clean.json'
jsonfile_healthOutcomes = 'Cleaned_Data/healthoutcome(2023).json'
#jsonfile_sig_vio ='Cleaned_Data/ECHO_Database_MN_SigViolations.geojson'

# create a list of json sources to loop through for collection builds
# if we add a data source above add it's var name to list below. 
sources = [jsonfile_cancer, jsonfile_asthma, jsonfile_epa, jsonfile_healthOutcomes]

#looping through list of sources and creating mongo collections
for json_file in sources:
    with open(json_file, 'r') as file:
        data = json.load(file)
        collection =db[(json_file.split('/')[-1].split('.')[0])]
        collection.insert_many(data)

#check to verity collections built
print(db.list_collection_names())

In [ ]:
# Step 3-Meta for collections for API developer
collection_names =db.list_collection_names()

for collection_name in collection_names:
        collection = db[collection_name]
        document = collection.find_one()
        print(f"\033[1mExample document from collection '{collection_name}'\n\n: \033[0m {document}\n\n")

In [ ]:
#Step 4-merging the data for the api easy grab

collection1 = db['cancer(2015-2019)']
collection2 = db['asthma(2018-2020)']
collection3 = db['healthoutcome(2023)']

# Function to merge documents from multiple collections
def merge_collections(source_collection, target_collection):
    for document in source_collection.find():
        county = document['County']
        document.pop('_id', None)
        target_collection.update_one({'County': county}, {'$set': document}, upsert=True)

#Merge documents from each collection into a new collection named 'merged_collection'
merged_collection = db['merged_collection']

merge_collections(collection1, merged_collection)
merge_collections(collection2, merged_collection)
merge_collections(collection3, merged_collection)

print("Merged collections successfully!")
#print an example for api developer.  
db['merged_collection'].find_one()

In [ ]:

# Close the MongoDB connection
client.close()